# 为指令数据集创建“被动语态”条目
- 本笔记本使用OpenAI的GPT-4生成“被动语态”条目，用于指令数据集，示例如下。

```python
{  
   'instruction': 'Identify the verb in the following sentence',
   'input': 'The cat sleeps on the couch.',
   'output': 'The verb in the sentence is "sleeps."',
   'output_2': 'The sentence is "sleeps."'   #  <---- Newly created entry
}  
```

In [2]:
pip install -r requirements-extra.txt

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ------ --------------------------------- 1.8/11.0 MB 9.2 MB/s eta 0:00:01
   ----------------------- ---------------- 6.6/11.0 MB 16.1 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/11.0 MB 16.3 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 14.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.8 MB ? eta -:--:--
   -- ------------------------------------- 2.9/44.8 MB 12.9 MB/s eta 0:00:04
   ------- -------------------------------- 8.1/44.8 MB 19.4 MB/s eta 0:00:02
   ---------- ----------------------------- 11.5/44.8 MB 18.0 MB/s eta 0:00:02
   ------------ --------------------------- 13.6/44.8 MB 15.8 MB/s eta 0:00:02
   ------------- -------------------------- 14.9/44.8 MB 14.0 MB/s eta 0:00:03
   --------------- ------------------------ 17.8/44.8 MB 15.2 MB/s eta 0:00:02
   ---------------- ----------------------- 18.1/44.8 MB 13.6 MB/s eta 0:00:

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
from importlib.metadata import version

pkgs = ["openai",  # OpenAI API
        "tqdm",    # Progress bar
       ]

for p in pkgs:
    print(f"{p} version: {version(p)}")

openai version: 1.54.4
tqdm version: 4.66.6


## 测试OpenAI API
- 首先，我们需要测试OpenAI API是否已正确设置。

- 如果你还没有账户，可以前往OpenAI平台创建一个。
- 请注意，GPT-4 API并不是免费的，你需要向账户转入资金（详见计费页面）。
- 使用本笔记本代码生成约200条“被动语态”条目需要花费约0.13美元（13美分）。

#### 提供API密钥
- 首先，需要提供你的OpenAI API密钥，该密钥可以在API密钥页面找到。
- 请确保不要将此密钥与他人分享。
- 将此密钥（以“sk-...”开头）添加到此文件夹中的config.json文件中。

In [ ]:
import json
from openai import OpenAI

# Load API key from a JSON file. 
# Make sure to replace "sk-..." with your actual API key from https://platform.openai.com/api-keys
with open("config.json", "r") as config_file:
    config = json.load(config_file)
    api_key = config["OPENAI_API_KEY"]

client = OpenAI(api_key=api_key)

In [ ]:
#First, let's try the API with a simple example to make sure it works as intended:
def run_chatgpt(prompt, client, model="gpt-4-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
    )
    return response.choices[0].message.content


# Prepare input
sentence = "I ate breakfast"
prompt = f"Convert the following sentence to passive voice: '{sentence}'"
run_chatgpt(prompt, client)

#### 创建JSON条目
接下来，我们加载需要修改的文件：



In [ ]:
import json

json_file = "instruction-examples.json"

with open(json_file, "r") as file:
    json_data = json.load(file)
    
print("Number of entries:", len(json_data))

In [ ]:
#我们会在一个小样本上尝试OpenAI的聊天API，以确保其能够正确运行。

for entry in json_data[:5]:
    text = entry["output"]
    prompt = f"Without adding any response or explanation, convert the following text to passive voice: {text}"
    
    print("\nInput:")
    print(">>", text)
    print("\nOutput:")
    print(">>", run_chatgpt(prompt, client))
    print("\n-------------------------")

- 现在，我们扩展代码，将生成的条目添加到json_data中，并加入一个进度条以监控过程。

In [ ]:
from tqdm import tqdm  # a progress bar tool


for i, entry in tqdm(enumerate(json_data[:5]), total=len(json_data[:5])):
    text = entry["output"]
    prompt = f"Without adding any response or explanation, convert the following text to passive voice: {text}"
    json_data[i]["output_2"] = run_chatgpt(prompt, client)

如果以上所有步骤运行正常，我们可以对整个JSON数据集执行被动语态转换（此过程大约需要3分钟）。

In [ ]:
for i, entry in tqdm(enumerate(json_data), total=len(json_data)):
    text = entry["output"]
    prompt = f"Without adding any response or explanation, convert the following text to passive voice: {text}"
    json_data[i]["output_2"] = run_chatgpt(prompt, client)

In [ ]:
#保存文件

new_json_file = json_file.replace(".json", "-modified.json")


with open(new_json_file, "w") as file:
    json.dump(json_data, file, indent=4)  # "indent" for pretty-printing